In [ ]:
from utils import *
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
import json
from loaders import *

In [ ]:
dv2_df = dreamerv2_loader()
ref_df = reference_loader()

In [ ]:
df = dv2_df.copy()
df = df.merge(ref_df, on="task")
df["norm"] = (df["score"] - df["random"]) / (df["human_gamer"] - df["random"])
g = df.groupby(["time"])
df = pd.concat({"mean": g["norm"].mean(), "median": g["norm"].median()}, axis=1)
df = df.reset_index()
df

In [ ]:
fig = make_subplots(
    cols=2,
    column_titles=["Human Mean", "Human Median"],
)

for trace in px.line(df, x="time", y="mean").data:
    fig.add_trace(trace, row=1, col=1)

for trace in px.line(df, x="time", y="median").data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(width=1000, height=400)
for col in (1, 2):
    fig.update_xaxes(title="Env step", type="log", row=1, col=col)
    fig.update_yaxes(title="Normalized score", row=1, col=col)

fig.write_image("../tex/assets/dreamerv2_curves.pdf")
fig

In [ ]:
from scipy.optimize import root_scalar


def f1(x, y):
    return np.interp(x, df["time"], df["mean"]) - y


def f2(x, y):
    return np.interp(x, df["time"], df["median"]) - y


print("Mean (Ours): ")
print(root_scalar(f1, args=(0.975,), x0=1e6, x1=6e6))

print("Median (Ours): ")
print(root_scalar(f2, args=(0.205,), x0=1e6, x1=6e6))

print("Median (BBF): ")
print(root_scalar(f2, args=(0.917,), x0=1e6, x1=6e6))